# 自製智能中文選字系統  (1)

確認版本為 python3

In [1]:
import sys
sys.version

'3.6.12 |Anaconda custom (64-bit)| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]'

## 資料前處理

In [2]:
import re

In [3]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    #pattern = r'[^\u4E00-\u9FA50-9]+'  # \u4e00-\u9fa5 : 中文字的unicode範圍
    pattern = r'[^\u4E00-\u9FFF]+'      # \u4e00-\u9fff : Unicode編碼範圍
    line = re.sub(pattern,' ', line)
    segments = line.strip().split(' ')
    #print(segments)
    # END YOUR CODE
    return segments

In [4]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [6]:
segments = []
with open('./wiki_zh_small.txt', encoding='utf-8') as fr:
    #i=0
    for line in fr.readlines():
        #print('>>> ' + str(i) + ': ' + line)
        segments += prepocess_line(line)
        #i+=1
    #print(i)

In [7]:
segments[:20]

['英語',
 '英語英語',
 '又稱爲英文',
 '是一種西日耳曼語言',
 '誕生於中世紀早期的英格蘭',
 '如今具有全球通用語的地位',
 '英語',
 '一詞源於遷居英格蘭的日耳曼部落盎格魯',
 '而',
 '盎格魯',
 '得名於臨波羅的海的半島盎格里亞',
 '弗裏西語是與英語最相近的語言',
 '英語詞彙在中世紀早期受到了其他日耳曼族語言的大量影響',
 '後來受羅曼族語言尤其是法語的影響',
 '英語是將近六十個國家唯一的官方語言或官方語言之一',
 '也是全世界最多國家的官方語言',
 '它是英國',
 '愛爾蘭',
 '美國',
 '加拿大']

## Ngram

一開始要先計算字詞出現的次數

In [8]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]  # 分別代表計算0、1、...個字的出現次數

    def fit(self, segments):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數
        for segment in segments:
            self.counters[0].update(['']*len(segment))
            for i in range(1,self.n+1):
                self.counters[i].update(self._skip(segment, i))

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [9]:
counters = Counters(n=3)
counters.fit(segments)

In [10]:
counters.counters[:]

[Counter({'': 371373}),
 Counter({'英': 420,
          '語': 1416,
          '又': 210,
          '稱': 724,
          '爲': 3637,
          '文': 1439,
          '是': 3600,
          '一': 3659,
          '種': 891,
          '西': 1266,
          '日': 1066,
          '耳': 49,
          '曼': 97,
          '言': 621,
          '誕': 22,
          '生': 1116,
          '於': 2022,
          '中': 3944,
          '世': 828,
          '紀': 320,
          '早': 211,
          '期': 769,
          '的': 13270,
          '格': 283,
          '蘭': 213,
          '如': 805,
          '今': 327,
          '具': 319,
          '有': 3252,
          '全': 994,
          '球': 278,
          '通': 659,
          '用': 1847,
          '地': 1756,
          '位': 701,
          '詞': 410,
          '源': 357,
          '遷': 89,
          '居': 282,
          '部': 1009,
          '落': 75,
          '盎': 16,
          '魯': 112,
          '而': 1055,
          '得': 599,
          '名': 824,
          '臨': 86,
          '波': 102,
      

In [11]:
counters[0]
# 應該為： Counter({'': 371373})

Counter({'': 371373})

In [12]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]
        
    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        # YOUR CODE HERE
        prefix = prefix[-(self.n-1):] if self.n > 1 else ''
        print(prefix)
        prefix_counts = self.minor_counter.get(prefix,0)
        print(prefix_counts)
        candidates = []
        for k, v in self.major_counter.items():
            if k.startswith(prefix):
                candidates.append((v/prefix_counts, k[-1]))
        
        sorted_probs = sorted(candidates, key=lambda x: x[0], reverse=True)
        # END YOUR CODE
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}


In [13]:
unigram = Ngram(1, counters)

In [14]:
unigram.predict_proba('我思')
# 應該為：[(0.035732269174118744, '的'),
#         (0.012927703414087723, '國'),
#         (0.010620050461395955, '中'),
#         (0.009984570768472667, '在'),
#         (0.009852627950874188, '一')]


371373


[(0.035732269174118744, '的'),
 (0.012927703414087723, '國'),
 (0.010620050461395955, '中'),
 (0.009984570768472667, '在'),
 (0.009852627950874188, '一')]

In [15]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

## 使用Ngram來建立第一版選字系統

In [16]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        # YOUR CODE HERE
        if not prefix:
            return self.unigram.predict_proba(prefix, top_k)
        elif len(prefix) == 1:
            return self.bigram.predict_proba(prefix, top_k)
        else:
            return self.trigram.predict_proba(prefix, top_k)
        # END YOUR CODE

In [17]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [18]:
probs = model.predict_proba('不列', top_k=10)
probs

不列
10


[(0.9, '顛'), (0.1, '入')]

## Demo

In [19]:
#!pip install -U pip
!pip install -q ipywidgets

In [20]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')